In [1]:
from database.database import Database

db = Database()
db.test_connection()

=================================CONFIG=================================
Database         User             Host                             Port            
citelinedb       bbasseri         localhost                        5432            
Database version: ('PostgreSQL 17.5 (Homebrew) on aarch64-apple-darwin24.4.0, compiled by Apple clang version 17.0.0 (clang-1700.0.13.3), 64-bit',)


In [5]:
# Set up Ollama service
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="deepseek-r1",
    temperature=0.0
)

ai_message = llm.invoke("What's your knowledge cutoff date?")
print(f"AI Message: {ai_message.content}")

AI Message: <think>

</think>

My knowledge is current up to December 2023. I'll do my best to provide accurate and comprehensive information for you.


In [8]:
import pandas as pd
research = pd.read_json('data/preprocessed/research.jsonl', lines=True, nrows=10)

paper = research.iloc[0]
print(f"Paper Title: {paper['title']}")
print(f"Paper Abstract: {paper['abstract']}")

Paper Title: The morphology of extragalactic radio sources of high and low luminosity
Paper Abstract: The relative positions of the high and low brightness regions in the extragalactic sources in the 3 CR complete sample are found to be correlated with the luminosity of these sources.


In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

prompt = """You are an expert research assistant. Identify the original contributions of the following research paper. Write the contributions as a JSON array of strings."""
system_prompt = SystemMessage(content=prompt)

full_paper_text = f"{paper['title']}\n{', '.join(paper['author'])}\n\nAbstract: {paper['abstract']}\n\n{paper['body']}"
print(f"Full Paper Text:\n{full_paper_text[:500]}...")  # Print first 500 characters for brevity



Full Paper Text:
The morphology of extragalactic radio sources of high and low luminosity
Fanaroff, B. L., Riley, J. M.

Abstract: The relative positions of the high and low brightness regions in the extragalactic sources in the 3 CR complete sample are found to be correlated with the luminosity of these sources.

Mon. Not. R. astr. Soc. (i~y~.) 167, Short Communication, 3 IP-35P. THE MORPHOLOGY OF EXTRAGALACTIC RADIO SOURCES OF HIGH AND LOW LUMINOSITY B. L. Fanaroff and J. M. Riley (Received 1974 March 6) SUMMA...


In [15]:
work_message = HumanMessage(content=full_paper_text)
response = llm.invoke([system_prompt, work_message])
print(f"Response: {response.content}")

Response: <think>
Okay, so I need to figure out the original contributions of this research paper about the morphology of extragalactic radio sources in terms of their high and low luminosity. The user has provided a detailed abstract and references, but they want me to identify the key contributions as JSON objects.

First, I'll read through the abstract carefully. It mentions that there's a correlation between the relative positions of high and low brightness regions in radio sources and their luminosity. They used the 3CR sample, which is a well-known dataset for radio galaxies. The authors classified these sources into two classes based on the ratio of distances between high and low brightness regions compared to the total extent.

I should note that they specifically analyzed sources with adequate resolution, excluding those where it was impossible or where classification wasn't feasible. This methodological approach seems like a significant contribution because it provides a clea